In [1]:
dseventhubs="ds-eventhubs"
dstwitters="ds-twitters"

print(dseventhubs) 
print(dstwitters) 


ds-eventhubs
ds-twitters

In [2]:
TwitterAPIkey=dbutils.secrets.get(scope=dstwitters, key="TwitterAPIkey")
TwitterAPISecret=dbutils.secrets.get(scope=dstwitters, key="TwitterAPISecret")
TwitterAccessToken=dbutils.secrets.get(scope=dstwitters, key="TwitterAccessToken")
TwitterAccessSecret=dbutils.secrets.get(scope=dstwitters, key="TwitterAccessSecret")

In [3]:
EventhubNamespace=dbutils.secrets.get(scope=dseventhubs, key="EventhubNamespace")
EventhubName=dbutils.secrets.get(scope=dseventhubs, key="EventhubName")
SharedAccessKeyName=dbutils.secrets.get(scope=dseventhubs, key="SharedAccessKeyName")
SharedAccessKey=dbutils.secrets.get(scope=dseventhubs, key="SharedAccessKey")

In [4]:
from azure.eventhub import EventHubClient, Sender, EventData
ADDRESS = "amqps://{}.servicebus.windows.net/{}".format(EventhubNamespace,EventhubName)


# Create Event Hubs client
client = EventHubClient(ADDRESS, debug=False, username=SharedAccessKeyName, password=SharedAccessKey)
sender = client.add_sender(partition="0")
client.run()

Out[ 7 ]: []

In [5]:
import tweepy

auth = tweepy.OAuthHandler(TwitterAPIkey, TwitterAPISecret)
auth.set_access_token(TwitterAccessToken, TwitterAccessSecret)

In [6]:
class MyStreamListener(tweepy.StreamListener):
        
    def on_status(self, status):
        print(status.text)
        message = "Message {}".format(status.text)
        sender.send(EventData(message))

api = tweepy.API(auth)
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [7]:
filter = dbutils.widgets.get("filter")
print(filter) 


In [8]:
myStream.filter(track=[filter])